In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
import gym
import vectorincrement
import numpy as np
import random
import gin
from vectorincrement import load_env
from vectorincrement.observation_encoder import KerasEncoder, KerasEncoderWrapper
from gym.wrappers import TimeLimit
from matplotlib import pyplot as plt
import tensorflow as tf

In [2]:
gin.enter_interactive_mode()
gin.parse_config_file('config/cartpole_linear.gin')

In [3]:
env = gym.make('VectorIncrement-v0')

In [5]:
env = load_env("VectorIncrement-v0", obs_scaler=0.1)

Make environment VectorIncrement-v0 None {}


In [6]:
env.reset()

0.1


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [7]:
env.step(0)

0.1


(array([0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ], dtype=float32),
 0.0,
 False,
 {})